### 화질 개선된 vid extract
- before bitrate : 200kbps, after : 55296kbps

In [ ]:
# get the test video from the repo
# extract for Maximum Quality(ffv1 codec 지원X)
#!yes | ffmpeg -ss 00:00:00 -i colon_video.avi -t 00:00:36 -vf fps=30,format=yuvj420p -c:v ffv1 colon_out.avi

# High bitrate command (00:00:52~00:01:40 # 영상시작시간~재생시간)  test_vid-4, 5, 8
!yes | ffmpeg -ss 00:03:24 -i output_raw_08.avi -t 00:00:33 -vf fps=30,format=yuvj420p -b:v 55296k polyp_test_8_gt_08-2_obj2_33s.avi

In [ ]:
import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

import os
os.getcwd()

### video tracking processing (deepoc, byte, bot)  

In [ ]:
# osnet_ain_x1_0_polyp_256x128_adam_ep205_stp20_lr0.003_triplet_labelsmooth.pt = osnet_ain_x1_0_polyp.pt

!python tracking/track.py --yolo-model yolov8m_polypPNG7.4_best.pt --tracking-method deepocsort --reid-model osnet_ain_x1_0_polyp.pt --source test_08_time_cent/test_08-30t40_obj2.avi --save --conf 0.82 --name yolov8m_polypPNG_deepocsort_08-30t40_obj2_triple_conf.82

In [ ]:
# deepocsort+lstm(MSE, 350e) - 비활성화
!python tracking/track.py --yolo-model yolov8m_polypPNG7.4_best.pt --tracking-method deepocsort --reid-model osnet_ain_x1_0_polyp.pt --source test_04_05_08_obj_centr/polyp_test_4_gt_04-3.avi --save --conf 0.82 --name polyp_test_4_gt_04-3_obj2_49s --save-id-crops

In [ ]:
# bytetrack은 reid model에 영향 x
!python tracking/track.py --yolo-model yolov8m_polypPNG7.4_best.pt --tracking-method bytetrack --reid-model osnet_ibn_x1_0_triplet.pt-160 --source test_08_time_cent/test_08-30t40_obj2.avi --save --conf 0.82 --name yolov8m_polypPNG__byte_08-30t40_obj2_triple_conf.82

In [ ]:
!python tracking/track.py --yolo-model yolov8m_polypPNG7.4_best.pt --tracking-method botsort --reid-model osnet_ain_x1_0_polyp.pt --source test_04_time_cent/test_04-80t40_obj1.avi --save --conf 0.82 --name yolov8m_polypPNG_botrsort+lstm_04-80t40_obj1_triple_conf.82

### avi -> mp4 quality 유지

In [ ]:
!ffmpeg -i colon_out_52to48.avi -vf fps=30 -vcodec libx264 -b:v 55296k -preset slow colon_output3.mp4

# Readme 기반으로 재작성한 실행 명령어

In [ ]:
# Track YOLO model
'''!python tracking/track.py --yolo-model '/home/kms2069/Projects/yolo_tracking/yolov8l_colon1_best.pt' --tracking-method deepocsort --reid-model osnet_x0_25_msmt17.pt --source colon_output.mp4 '''

In [ ]:
# saves dets and embs under ./runs/dets_n_embs separately for each selected yolo and reid model
# 선택한 각 yolo & reid model에 대해 dets & embs를 ./runs~에 별도 저장(현재: home\kms2069\anaconda3\envs\boxmot\lib\python3.8\site-packages\runs)
# file 덮어쓰기 주의!
!PYTHONPATH=. python tracking/generate_dets_n_embs.py --source ./assets/polyp_test_mot/train --yolo-model yolov8m_polypPNG7.4_best.pt --reid-model osnet_ain_x1_0_polyp.pt --project ./runs/dets_n_embs/ --name yolov8m_polypPNG7.4_2

In [ ]:
!python tracking/generate_dets_n_embs.py --source ./assets/colon_mot_trainall_test_608_1274/train --yolo-model '/home/kms2069/Projects/yolo_tracking/yolov8s_colon1_best.pt' --reid-model osnet_x1_0.pt-200

In [ ]:
!python tracking/generate_dets_n_embs.py --source ./assets/colon_mot_3/train --yolo-model '/home/kms2069/Projects/yolo_tracking/yolov8s_colon1_best.pt' --reid-model osnet_x0_25_msmt17.pt

In [ ]:
# generate MOT challenge format results based on pregenerated detections and embeddings for a specific tracking method
# 특정 tracking방법에 대해 사전생성된 detection 및 embbedings를 기반으로 MOT challenge 형식 결과 생성
# custom reid-model 추가
!PYTHONPATH=. python tracking/generate_mot_results.py --dets yolov8m_polypPNG7.4_best --embs osnet_ain_x1_0_polyp --tracking-method deepocsort --reid-model osnet_ain_x1_0_polyp.pt 

In [ ]:
!python tracking/generate_mot_results.py --dets yolov8s_colon1_best --embs osnet_x1_0 --tracking-method botsort

In [ ]:
!python tracking/generate_mot_results.py --dets yolov8s_colon1_best --embs osnet_x0_25_msmt17 --tracking-method botsort

In [ ]:
!PYTHONPATH=. python tracking/val.py --yolo-model yolov8m_polypPNG7.4_best.pt --benchmark polyp_test_mot --dets yolov8m_polypPNG7.4_best --embs osnet_ain_x1_0_polyp --tracking-method deepocsort 

In [ ]:
!PYTHONPATH=. python tracking/val.py --yolo-model yolov8m_polypPNG7.4_best.pt --reid-model osnet_ain_x1_0_polyp.pt --tracking-method deepocsort --verbose --source ./assets/polyp_test_mot/train --save-txt --save-mot --save-id-crops

# osnet_ain_x1_0_polyp.pt, osnet_x0_25_msmt17.pt
# --verbose --source ./tracking/val_utils/MOT17/train
# --no-fps-check

In [ ]:
!PYTHONPATH=. python tracking/val.py --benchmark colon_mot_3 --dets yolov8s_colon1_best --embs osnet_x0_25_msmt17 --tracking-method botsort

In [ ]:
!python tracking/evolve.py --benchmark polyp_mot --dets yolov8n_colon1_best --embs osnet_x0_25_msmt17 --n-trials 9 --tracking-method botsort
